## Linux Scraper

This Notebook serves the purpose to provide the tools to do scraping on my linux machine. 
It enables me to collect data with my main windows laptop, but also with my small but powerfull linux dekstop.

It is also the first step to running automatized tasks on my linux machine using crontab to automatically scrape the data weekly.

In [1]:
import urllib.request, json , time, os, difflib, itertools
import pandas as pd
import numpy as np
from multiprocessing.dummy import Pool
from datetime import datetime
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from pandas import ExcelWriter
from bs4 import BeautifulSoup
from scipy import stats #The SciPy stats module
import datetime



def get_stock_data(url, country_name):
    browser =  webdriver.Firefox(executable_path='/home/jonathan/Schreibtisch/geckodriver')
    time.sleep(2)
    browser.get(url)
    time.sleep(5)
    soup = BeautifulSoup(browser.page_source,"html")
    time.sleep(5)
    names = []
    Row = soup.find('table', attrs = {'id' : 'insert-stocks'})
    for name in Row.find_all('td', attrs = {'class' : 'name'}):
            try:
                names.append(name.text)
            except:
                names.append("")

    wkns = []
    Row = soup.find('table', attrs = {'id' : 'insert-stocks'})
    for wkn in Row.find_all('td', attrs = {'data-id' : 'wkn'}):
            try:
                wkns.append(wkn.text)
            except:
                wkns.append("")

    FRAME = pd.DataFrame.from_dict(
            {'name': names,
            'wkn': wkns,
            'index' : country_name
            })
    time.sleep(1)
    browser.quit()
    return(FRAME)


def get_ticker(initial_frame):
    url_ticker = "https://www.finanznachrichten.de/"
    url_yf_ticker = "https://finance.yahoo.com/quote/FB?p=FB"
    browser =  webdriver.Firefox(executable_path='/home/jonathan/Schreibtisch/geckodriver')
    extension_path = r"/home/jonathan/.mozilla/firefox/5xapxbqn.default-release/extensions/jid1-MnnxcxisBPnSXQ@jetpack.xpi"
    browser.install_addon(extension_path, temporary=True)
    time.sleep(2)
    #extension_path = r"C:\Users\Jonathan\AppData\Roaming\Mozilla\Firefox\Profiles\wl4weym2.default-1633941918487\extensions\firefox-webext@zenmate.com.xpi"
    #browser.install_addon(extension_path, temporary=True)
    browser.maximize_window()
    browser.get("about:support")
    time.sleep(5)
    browser.get(url_ticker)
    time.sleep(10)
    #clear = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/div[3]/div[2]/button"))).click()
    time.sleep(1)
    ticker = []
    de_ticker = []
    industry = []
    ISIN = []
    for i, wkn in enumerate(initial_frame.wkn):
        print(len(de_ticker), len(industry), len(ISIN))
        # if wkn not valid
        if wkn == "-":
            de_ticker.append("not_found")
            industry.append("not_found")
            ISIN.append("not_found")
            continue
        try:
            time.sleep(1)
            clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe"))).clear()
            time.sleep(random.randint(3,5))
            search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#fnk-suche-eingabe")))
            time.sleep(random.randint(2,3))
            search.send_keys(str(wkn))
            time.sleep(4)
            browser.find_element_by_css_selector('#suchhilfeListe > tbody:nth-child(2) > tr:nth-child(3) > td:nth-child(2)').click()
            time.sleep(random.randint(3,5))
            de_ticker.append(str(WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-ticker"))).text.replace("Ticker-Symbol: ", "").lstrip()))
            time.sleep(2)
        except:
            de_ticker.append("not_found")
            industry.append("not_found")
            ISIN.append("not_found")
            continue
        try:
            industry.append(str(WebDriverWait(browser, 19).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".a > a:nth-child(2)"))).text))
            #time.sleep(3)
        except:
            industry.append("not_found")
        try:
            ISIN.append(str(WebDriverWait(browser, 19).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#produkt-isin"))).text.replace("ISIN: ", "").lstrip()))
            #time.sleep(random.randint(2,4))
        except:
            ISIN.append("not_found")


        if i % 4 == 0:
            try:
                browser.refresh()
                time.sleep(random.randint(4,5))
            except:
                pass

    #check if lengths are equal   
    print(len(industry), len(ISIN), len(de_ticker))

    browser.get(url_yf_ticker)
    time.sleep(3)
    try:
        browser.find_element_by_css_selector("#scroll-down-btn").click()
        time.sleep(3)
    except:
        pass
    try:
        browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
        time.sleep(4)
    except:
        pass
    for _ISIN in ISIN:
        print(_ISIN)
        if _ISIN == "not_found":
            ticker.append("not_found")
            continue
        time.sleep(random.randint(1,4))
        #check again for popup and scroll
        try:
            browser.find_element_by_css_selector("#scroll-down-btn").click()
            time.sleep(3)
        except:
            pass
        try:
            browser.find_element_by_css_selector("button.btn:nth-child(5)").click()
            time.sleep(4)
        except:
            pass

        clear = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry"))).clear()
        time.sleep(random.randint(3,4))
        try:
            search = WebDriverWait(browser, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#yfin-usr-qry")))
            time.sleep(1)
            search.send_keys(str(_ISIN))
            time.sleep(2)
            ticker.append(str(WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#result-quotes-0 > div.modules_quoteLeftCol__gkCSv.modules_Ell__77DLP.modules_IbBox__2pmLe > div.modules_quoteSymbol__hpPcM.modules_Ell__77DLP.modules_IbBox__2pmLe"))).text))
        except:
            ticker.append("not_found")
            
    print(len(ticker))      

    frame = pd.DataFrame.from_dict(
        {"name" : initial_frame.name,
        "wkn" : initial_frame.wkn,
        "index" : "NYSE",
        'de_ticker' : de_ticker,
        'ticker': ticker,
        'industry': industry,
        'ISIN': ISIN
        })
    time.sleep(1)
    browser.quit()
    return frame

In [ ]:
url_ls = ["https://traderfox.de/aktien/alle-nyse-aktien-bestandteile"]#,
   # "https://traderfox.de/aktien/deutschland-160-bestandteile",
   # "https://traderfox.de/aktien/alle-nyse-aktien-bestandteile",
  #  "https://traderfox.de/aktien/alle-nasdaq-aktien-bestandteile",
#]
#https://traderfox.de/aktien/stoxx-europe-600-bestandteile
country_names_ls = ["NYSE" ,"NASDAQ"]

ticker_frame = pd.DataFrame({})
for i, url in enumerate(url_ls):
    df = get_stock_data(url, country_names_ls[i])
    ticker_frame = ticker_frame.append(df)
    time.sleep(1)

ticker_frame

In [ ]:
data_ticker_nyse = get_ticker(ticker_frame[1675:1680])
data_ticker_nyse.to_csv("NYSEStocks_p3.csv")